# Token Generation & Security Procedures - Practical Learning

This notebook teaches you how to write SQL procedures similar to `mobile_bearer_token_get.sql`.

**Learning Goals:**
- Random token generation techniques
- Output parameters usage
- Database cleanup operations
- Error handling and status codes
- Security best practices

**Prerequisites:** Basic SQL knowledge and stored procedure concepts.

## Setup: Create Practice Tables

First, let's create the tables we'll use for practicing token generation procedures.

In [ ]:
-- Create practice schema and tables
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'practice')
BEGIN
    EXEC('CREATE SCHEMA practice');
END
GO

-- Create token master table
IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'token_master' AND schema_id = SCHEMA_ID('practice'))
BEGIN
    CREATE TABLE practice.token_master (
        id INT IDENTITY(1,1) PRIMARY KEY,
        token_id VARCHAR(32) NOT NULL UNIQUE,
        created_datetime DATETIME2 DEFAULT GETDATE(),
        expire_datetime DATETIME2 DEFAULT DATEADD(HOUR, 24, GETDATE()),
        is_active BIT DEFAULT 1
    );
END

SELECT 'Tables created successfully' AS Status;

## Exercise 1: Basic Random String Generation

**Task:** Learn how to generate random strings using T-SQL.

**Concept:** Understanding NEWID(), CHECKSUM(), and string manipulation functions.

In [ ]:
-- Exercise 1: Basic Random String Generation

-- Step 1: Understand NEWID() and CHECKSUM()
SELECT 
    NEWID() AS RandomGUID,
    CHECKSUM(NEWID()) AS RandomNumber,
    ABS(CHECKSUM(NEWID())) AS PositiveRandomNumber;

-- Step 2: Generate single random character
DECLARE @char_set VARCHAR(62) = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789';
DECLARE @random_position INT = ABS(CHECKSUM(NEWID())) % LEN(@char_set) + 1;

SELECT 
    @char_set AS CharacterSet,
    LEN(@char_set) AS SetLength,
    @random_position AS RandomPosition,
    SUBSTRING(@char_set, @random_position, 1) AS RandomCharacter;

-- Step 3: Generate 8-character random string
DECLARE @result VARCHAR(8) = '';
DECLARE @i INT = 1;

WHILE @i <= 8
BEGIN
    SET @result = @result + SUBSTRING(@char_set, ABS(CHECKSUM(NEWID())) % LEN(@char_set) + 1, 1);
    SET @i = @i + 1;
END

SELECT @result AS RandomString8Chars;

## Exercise 2: Simple Token Generator Function

**Task:** Create a reusable function to generate random tokens.

**Concept:** Encapsulating token generation logic in a scalar function.

In [ ]:
-- Exercise 2: Create Token Generator Function

CREATE OR ALTER FUNCTION practice.GenerateRandomToken(@Length INT)
RETURNS VARCHAR(64)
AS
BEGIN
    DECLARE @token VARCHAR(64) = '';
    DECLARE @char_set VARCHAR(62) = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789';
    DECLARE @i INT = 1;
    
    -- Validate input
    IF @Length <= 0 OR @Length > 64
        SET @Length = 32; -- Default length
    
    -- Generate random string
    WHILE @i <= @Length
    BEGIN
        SET @token = @token + SUBSTRING(@char_set, ABS(CHECKSUM(NEWID())) % LEN(@char_set) + 1, 1);
        SET @i = @i + 1;
    END
    
    RETURN @token;
END;
GO

-- Test the function
SELECT 
    practice.GenerateRandomToken(8) AS Token8,
    practice.GenerateRandomToken(16) AS Token16,
    practice.GenerateRandomToken(32) AS Token32,
    practice.GenerateRandomToken(0) AS TokenDefault; -- Should return 32 chars

## Exercise 3: Complete Token Management Procedure

**Task:** Create a comprehensive procedure similar to `mobile_bearer_token_get.sql`.

**Concept:** Combining generation, cleanup, error handling, and status codes.

In [ ]:
-- Exercise 3: Complete Token Management Procedure (Similar to mobile_bearer_token_get)

CREATE OR ALTER PROCEDURE practice.mobile_bearer_token_get
    @status_code INT = 0 OUTPUT,
    @status_message VARCHAR(200) = '' OUTPUT
AS
BEGIN
    SET NOCOUNT ON;
    
    -- Declare variables (same as original procedure)
    DECLARE @token VARCHAR(32) = '';
    DECLARE @char_set VARCHAR(75) = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789';
    DECLARE @Length INT = 32;
    DECLARE @token_id VARCHAR(32) = '';
    DECLARE @i INT = 1;
    
    BEGIN TRY
        -- Generate random token (same logic as original)
        WHILE @i <= @Length
        BEGIN
            SET @token_id = @token_id + SUBSTRING(@char_set, ABS(CHECKSUM(NEWID())) % LEN(@char_set) + 1, 1);
            SET @i = @i + 1;
        END
        
        -- Cleanup expired tokens (same as original)
        DELETE FROM practice.token_master 
        WHERE expire_datetime < GETDATE();
        
        -- Insert new token (same as original)
        INSERT INTO practice.token_master (token_id)
        VALUES (@token_id);
        
        -- Check if insertion was successful (same logic as original)
        IF @@ROWCOUNT > 0
        BEGIN
            SELECT @status_code = 200;
            SELECT @status_message = 'Created';
        END
        ELSE
        BEGIN
            SELECT @token_id = '';
            SELECT @status_code = 400;
            SELECT @status_message = 'Token Generation Failed';
        END
        
    END TRY
    BEGIN CATCH
        -- Handle any errors
        SELECT @token_id = '';
        SELECT @status_code = 500;
        SELECT @status_message = 'Internal Error: ' + ERROR_MESSAGE();
    END CATCH
    
    -- Return results (same format as original)
    SELECT 
        @token_id AS token_id,
        @status_code AS status_code,
        @status_message AS status_message;
END;
GO

-- Test the complete procedure
DECLARE @code INT, @message VARCHAR(200);

EXEC practice.mobile_bearer_token_get
    @status_code = @code OUTPUT,
    @status_message = @message OUTPUT;

PRINT 'Status Code: ' + CAST(@code AS VARCHAR(10));
PRINT 'Status Message: ' + @message;

-- View all tokens in the table
SELECT * FROM practice.token_master ORDER BY created_datetime DESC;

## Practice Exercises

Try these exercises to master token generation procedures:

1. **Modify Token Length**: Change the procedure to accept a parameter for token length
2. **Add Expiration Control**: Add a parameter to set custom expiration time
3. **Token Validation**: Create a procedure to validate if a token exists and is active
4. **Bulk Token Generation**: Create a procedure that generates multiple tokens at once
5. **Token Statistics**: Create a procedure that returns token usage statistics

## Key Concepts Summary

**Random Generation:**
- `NEWID()` creates random GUIDs
- `CHECKSUM()` converts to numbers
- `ABS()` ensures positive values
- Modulo `%` operator for range control

**Output Parameters:**
- Use `OUTPUT` keyword in parameter declaration
- Assign values using `SELECT @param = value`
- Return multiple values from procedures

**Database Operations:**
- `@@ROWCOUNT` checks affected rows
- `DELETE` for cleanup operations
- `INSERT` for new records

**Error Handling:**
- `TRY-CATCH` blocks for error management
- Status codes for API responses
- Meaningful error messages